## PRE-CONFIGURACIÓN

In [1]:
!pip install -q torch torchvision torchaudio

In [2]:
import torch
print("PyTorch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())

PyTorch version: 2.5.1+cu121
GPU available: True


## INSTALACIÓN

In [3]:
!pip install -q transformers bitsandbytes accelerate

In [4]:
from dataclasses import dataclass, field
from collections import Counter
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import traceback
import time
import json
import csv
import os
import re

## MODELO: QWEN

In [5]:
MODEL_ID = "Qwen/Qwen3-8B"

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

class QWen:
    def __init__(self):
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4"
        )

        self.tokenizer = AutoTokenizer.from_pretrained(
            MODEL_ID,
            use_fast=True,
            trust_remote_code=True
        )

        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            quantization_config=bnb_config,
            device_map="auto",
            dtype=torch.float16,
        )

        self.model.eval()

        print(f"Modelo QWen cargado correctamente en {self.model.device}")

    def execute(self, prompt:str, system:str=None, temperature:float=0.3, max_tokens:int=512, top_p:int=0.9) -> str:
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
        messages.append({"role": "user", "content": prompt})

        text = self.tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
        inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            out = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.eos_token_id,
            )

        gen_ids = out[0, inputs.input_ids.shape[1] :]

        return self.tokenizer.decode(gen_ids, skip_special_tokens=True)

In [7]:
qwen = QWen()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Modelo QWen cargado correctamente en cuda:0


## ESTRATEGIAS

### Base

In [8]:
HEADERS_RESULTS = [
    "index",
    "id",
    
    "intent_predicted",
    "intent_example",
    "intent_is_consistent",
    
    "priority_predicted",
    "priority_example",
    "priority_is_consistent",
    
    "route_team_predicted",
    "route_team_example",
    "route_team_is_consistent",
    
    "customer_tier_predicted",
    "customer_tier_example",
    "customer_tier_is_consistent",
    
    "impact_predicted",
    "impact_example",
    "impact_is_consistent",
    
    "downtime_minutes_predicted",
    "downtime_minutes_example",
    "downtime_minutes_is_consistent",
    
    "payment_risk_predicted",
    "payment_risk_example",
    "payment_risk_is_consistent",
    
    "has_sensitive_data_predicted",
    "has_sensitive_data_example",
    "has_sensitive_data_is_consistent",
    
    "critical_feature_blocked_predicted",
    "critical_feature_blocked_example",
    "critical_feature_blocked_is_consistent",
    
    "language_predicted",
    "language_example",
    "language_is_consistent",
    
    "action_plan_steps_predicted",
    "action_plan_steps_example",
    "action_plan_steps_is_consistent",
    
    "action_plan_predicted",
    "action_plan_example",

    "notify_count_predirect",
    "notify_count_example",
    "notify_count_is_consistent",
    
    "notify_predicted",
    "notify_example",

    "processing_time_ms",
    
    "is_business_rules_consistent",
    
    "error",
]

HEADERS_RESULTS_COT_SC = [
    "index",
    "id",
    
    "intent_predicted",
    "intent_example",
    "intent_is_consistent",
    
    "priority_predicted",
    "priority_example",
    "priority_is_consistent",
    
    "route_team_predicted",
    "route_team_example",
    "route_team_is_consistent",
    
    "customer_tier_predicted",
    "customer_tier_example",
    "customer_tier_is_consistent",
    
    "impact_predicted",
    "impact_example",
    "impact_is_consistent",
    
    "downtime_minutes_predicted",
    "downtime_minutes_example",
    "downtime_minutes_is_consistent",
    
    "payment_risk_predicted",
    "payment_risk_example",
    "payment_risk_is_consistent",
    
    "has_sensitive_data_predicted",
    "has_sensitive_data_example",
    "has_sensitive_data_is_consistent",
    
    "critical_feature_blocked_predicted",
    "critical_feature_blocked_example",
    "critical_feature_blocked_is_consistent",
    
    "language_predicted",
    "language_example",
    "language_is_consistent",
    
    "action_plan_steps_predicted",
    "action_plan_steps_example",
    "action_plan_steps_is_consistent",
    
    "action_plan_predicted",
    "action_plan_example",

    "notify_count_predirect",
    "notify_count_example",
    "notify_count_is_consistent",
    
    "notify_predicted",
    "notify_example",

    "trials",
    "trial_winner",

    "processing_time_ms",
    
    "is_business_rules_consistent",
    
    "error",
]

HEADERS_RESULTS_TOT = [
    "index",
    "id",
    
    "intent_predicted",
    "intent_example",
    "intent_is_consistent",
    
    "priority_predicted",
    "priority_example",
    "priority_is_consistent",
    
    "route_team_predicted",
    "route_team_example",
    "route_team_is_consistent",
    
    "customer_tier_predicted",
    "customer_tier_example",
    "customer_tier_is_consistent",
    
    "impact_predicted",
    "impact_example",
    "impact_is_consistent",
    
    "downtime_minutes_predicted",
    "downtime_minutes_example",
    "downtime_minutes_is_consistent",
    
    "payment_risk_predicted",
    "payment_risk_example",
    "payment_risk_is_consistent",
    
    "has_sensitive_data_predicted",
    "has_sensitive_data_example",
    "has_sensitive_data_is_consistent",
    
    "critical_feature_blocked_predicted",
    "critical_feature_blocked_example",
    "critical_feature_blocked_is_consistent",
    
    "language_predicted",
    "language_example",
    "language_is_consistent",
    
    "action_plan_steps_predicted",
    "action_plan_steps_example",
    "action_plan_steps_is_consistent",
    
    "action_plan_predicted",
    "action_plan_example",

    "notify_count_predirect",
    "notify_count_example",
    "notify_count_is_consistent",
    
    "notify_predicted",
    "notify_example",

    "beam_width",
    "final_depth",
    "final_score",

    "processing_time_ms",
    
    "is_business_rules_consistent",
    
    "error",
]

HEADERS_RESULTS_GOT = [
    "index",
    "id",
    
    "intent_predicted",
    "intent_example",
    "intent_is_consistent",
    
    "priority_predicted",
    "priority_example",
    "priority_is_consistent",
    
    "route_team_predicted",
    "route_team_example",
    "route_team_is_consistent",
    
    "customer_tier_predicted",
    "customer_tier_example",
    "customer_tier_is_consistent",
    
    "impact_predicted",
    "impact_example",
    "impact_is_consistent",
    
    "downtime_minutes_predicted",
    "downtime_minutes_example",
    "downtime_minutes_is_consistent",
    
    "payment_risk_predicted",
    "payment_risk_example",
    "payment_risk_is_consistent",
    
    "has_sensitive_data_predicted",
    "has_sensitive_data_example",
    "has_sensitive_data_is_consistent",
    
    "critical_feature_blocked_predicted",
    "critical_feature_blocked_example",
    "critical_feature_blocked_is_consistent",
    
    "language_predicted",
    "language_example",
    "language_is_consistent",
    
    "action_plan_steps_predicted",
    "action_plan_steps_example",
    "action_plan_steps_is_consistent",
    
    "action_plan_predicted",
    "action_plan_example",

    "notify_count_predirect",
    "notify_count_example",
    "notify_count_is_consistent",
    
    "notify_predicted",
    "notify_example",

    "nodes_executed",
    "backtracking_applied",
    "consistency_check",
    "execution_log",

    "processing_time_ms",
    
    "is_business_rules_consistent",
    
    "error",
]

SYSTEM_PROMPT = """
    Eres un sistema experto en triage de tickets de soporte.
    Debes procesar tickets (en español o inglés, con ruido) y generar un análisis completo aplicando reglas de negocio.

    ENTIDADES A EXTRAER DEL TICKET:
    1) intent: security | outage | login | billing | cancellation | bug_report | feature_request
    2) customer_tier: free | standard | enterprise
    3) impact: single_user | multi_users_same_team | org_wide
    4) downtime_minutes: número entero (minutos de inactividad)
    5) payment_risk: low | medium | high
    6) has_sensitive_data: true | false (contiene PII, credenciales, datos sensibles)
    7) critical_feature_blocked: true | false (bloquea funcionalidad crítica del sistema)
    8) language: es | en (idioma del ticket)

    REGLAS DE NEGOCIO - PRIORIDAD (P1-P4):
    Aplica estas reglas EN ORDEN para determinar la prioridad base:

    • P1: Si intent = security, O si intent = outage AND impact = org_wide, O si downtime_minutes > 120
    • P2: Si intent = outage AND downtime_minutes está entre 30 y 120, O si intent = login AND impact = multi_users_same_team, O si payment_risk = high
    • P3: Si intent = billing AND payment_risk ≠ high, O si intent = bug_report AND critical_feature_blocked = true, O si intent = cancellation AND customer_tier = enterprise
    • P4: En todos los demás casos

    ELEVACIÓN DE PRIORIDAD:
    • Si has_sensitive_data = true, ELEVAR la prioridad un nivel: P4→P3, P3→P2, P2→P1 (P1 se mantiene como P1)

    REGLAS DE RUTEO (route_team según intent):
    • security → security_ir
    • outage → infra_oncall
    • login → auth_platform
    • billing → billing_ops
    • cancellation → billing_ops
    • bug_report → app_backend
    • feature_request → product_mgmt

    REGLAS DE NOTIFICACIÓN:
    • Si intent = cancellation AND customer_tier = enterprise, entonces notify = ["cx_retention"]
    • En todos los demás casos, notify = []

    PLAN DE ACCIÓN:
    • Debe contener entre 3 y 6 pasos
    • Debe redactarse en el MISMO idioma del ticket (español si language=es, inglés si language=en)
    • Si has_sensitive_data = true, el plan DEBE incluir pasos de sanitización de datos y notificación de privacidad

    FORMATO DE SALIDA - Retorna ÚNICAMENTE un JSON válido (sin texto adicional):
    {
        "intent": "security",
        "priority": "P1",
        "route_team": "security_ir",
        "notify": [],
        "entities": {
            "customer_tier": "enterprise",
            "impact": "org_wide",
            "downtime_minutes": 0,
            "payment_risk": "low",
            "has_sensitive_data": false,
            "critical_feature_blocked": false,
            "language": "es"
        },
        "action_plan": [
            "Paso 1...",
            "Paso 2...",
            "Paso 3..."
        ]
    }
"""

In [9]:
def clean_json_response(response: str) -> Dict:
    response = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL)
    response = re.sub(r"```json\s*", "", response)
    response = re.sub(r"```\s*$", "", response)

    start = response.find("{")
    end = response.rfind("}")

    if start != -1 and end != -1 and end > start:
        json_str = response[start : end + 1].strip()
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Failed to parse JSON: {e}")

    raise ValueError("No JSON object found in response")

In [10]:
def load_jsonl(filepath: str) -> List[Dict]:
    data = []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

In [11]:
def init_csv_results(strategy_name: str) -> str:
    os.makedirs("results", exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_path = f"results/{strategy_name}_{timestamp}.csv"
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter="|")
        headers = HEADERS_RESULTS
        if strategy_name == "cot_sc":
            headers = HEADERS_RESULTS_COT_SC
        elif strategy_name == "tot":
            headers = HEADERS_RESULTS_TOT
        elif strategy_name == "got":
            headers = HEADERS_RESULTS_GOT
        writer.writerow(headers)
    return csv_path

In [12]:
def save_line_in_csv_results(csv_path: str, index: int, id: str, example, result, processing_time: float, error: str = None, strategy_name: str = None) -> str:
    consistents = {
        "intent": example["intent"] == result["intent"],
        "priority": example["priority"] == result["priority"],
        "route_team": example["route_team"] == result["route_team"],
        "customer_tier": example["customer_tier"] == result["customer_tier"],
        "impact": example["impact"] == result["impact"],
        "downtime_minutes": example["downtime_minutes"] == result["downtime_minutes"],
        "payment_risk": example["payment_risk"] == result["payment_risk"],
        "has_sensitive_data": example["has_sensitive_data"] == result["has_sensitive_data"],
        "critical_feature_blocked": example["critical_feature_blocked"] == result["critical_feature_blocked"],
        "language": example["language"] == result["language"],
        "action_plan_steps": len(result["action_plan_steps"]) >= 3 or len(result["action_plan_steps"]) <= 6,
        "notify": len(example["notify"]) == len(result["notify"]),
    }
    is_consistent = all(v for v in consistents.values() if isinstance(v, bool))
    
    row = [
        index,
        id,

        result["intent"],
        example["intent"],
        consistents["intent"],
        
        result["priority"],
        example["priority"],
        consistents["priority"],
        
        result["route_team"],
        example["route_team"],
        consistents["route_team"],
        
        result["customer_tier"],
        example["customer_tier"],
        consistents["customer_tier"],
        
        result["impact"],
        example["impact"],
        consistents["impact"],
        
        result["downtime_minutes"],
        example["downtime_minutes"],
        consistents["downtime_minutes"],
        
        result["payment_risk"],
        example["payment_risk"],
        consistents["payment_risk"],
        
        result["has_sensitive_data"],
        example["has_sensitive_data"],
        consistents["has_sensitive_data"],
        
        result["critical_feature_blocked"],
        example["critical_feature_blocked"],
        consistents["critical_feature_blocked"],
        
        result["language"],
        example["language"],
        consistents["language"],

        len(result["action_plan_steps"]),
        len(example["action_plan_steps"]),
        consistents["action_plan_steps"],
        
        json.dumps(result["action_plan_steps"], ensure_ascii=False),
        json.dumps(example["action_plan_steps"], ensure_ascii=False),

        len(result["notify"]),
        len(example["notify"]),
        consistents["notify"],
        
        json.dumps(result["notify"], ensure_ascii=False),
        json.dumps(example["notify"], ensure_ascii=False),
    ]

    if strategy_name == "cot_sc":
        row.append(result["trials"])
        row.append(result["trial_winner"])
    elif strategy_name == "tot":
        row.append(result["beam_width"])
        row.append(result["final_depth"])
        row.append(result["final_score"])
    elif strategy_name == "got":
        row.append(result["nodes_executed"])
        row.append(result["backtracking_applied"])
        row.append(json.dumps(result["consistency_check"]))
        row.append(json.dumps(result["execution_log"]))
    
    row.append(processing_time)
    row.append(is_consistent)
    row.append(error)
    
    with open(csv_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter="|")
        writer.writerow(row)

In [15]:
DEV_DATA = load_jsonl("data/dev.jsonl")
TRAIN_DATA = load_jsonl("data/train.jsonl")
TEST_PUBLIC__DATA = load_jsonl("data/test_public.jsonl")
TEST_HIDDEN_DATA = load_jsonl("data/test_hidden.jsonl")

In [16]:
class BaseStrategy:
    def __init__(self, qwen):
        self.qwen = qwen

    @staticmethod
    def calculate_expected_priority(
        intent: str,
        impact: str,
        downtime_minutes: float,
        payment_risk: str,
        critical_feature_blocked: bool,
        customer_tier: str,
        has_sensitive_data: bool,
    ) -> str:
        priority = 0
        # P1
        if (
            intent == "security"
            or (intent == "outage" and impact == "org_wide")
            or downtime_minutes > 120
        ):
            priority = 1
        # P2
        elif (
            (intent == "outage" and 30 <= downtime_minutes <= 120)
            or (intent == "login" and impact == "multi_users_same_team")
            or payment_risk == "high"
        ):
            priority = 2
        # P3
        elif (
            (intent == "billing" and payment_risk != "high")
            or (intent == "bug_report" and critical_feature_blocked)
            or (intent == "cancellation" and customer_tier == "enterprise")
        ):
            priority = 3
        # P4 (default)
        else:
            priority = 4

        # Si has_sensitive_data=true -> Elevar 1 nivel
        if has_sensitive_data and priority > 1:
            priority = priority - 1

        return f"P{priority}"

    @staticmethod
    def calculate_expected_route(intent: str) -> str:
        routes = {
            "security": "security_ir",
            "outage": "infra_oncall",
            "login": "auth_platform",
            "billing": "billing_ops",
            "cancellation": "billing_ops",
            "bug_report": "app_backend",
            "feature_request": "product_mgmt",
        }
        return routes.get(intent, "unknown")

    @staticmethod
    def calculate_expected_notify(intent, customer_tier) -> List[str]:
        notify = []
        if intent == "cancellation" and customer_tier == "enterprise":
            notify.append("cx_retention")
        return notify

### Zero Shot

In [17]:
class ZeroShot(BaseStrategy):
    def __init__(self, qwen: QWen):
        super().__init__(qwen)
        self.params = {
            "temperature": 0.2,
            "max_tokens": 3000,
        }

    def build_prompt(self, ticket_text: str) -> str:
        prompt = f"""
            Devuelve el resultado de análisis de este ticket.
            Ticket:
            {ticket_text}
        """
        return prompt

    def execute(self, ticket_text: str) -> Dict:
        prompt = self.build_prompt(ticket_text=ticket_text)
        response = self.qwen.execute(
            prompt=prompt,
            system=SYSTEM_PROMPT,
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
        )
        return clean_json_response(response)

In [18]:
zero_shot_strategy = ZeroShot(qwen)

In [21]:
def zero_shot_execute(dataset, datatype: str):
    zero_shot_csv_path = init_csv_results(f"zero_shot_{datatype}")
    print(f"Los resultados se guardarán en: {zero_shot_csv_path}")
    
    print(f"Procesando tickets con Zero Shot...")
    start_time = time.time()
    for i, example in enumerate(dataset):
        print(f"\n[{i+1}/{len(dataset)}] Processing ticket...")
        local_start_time = time.time()
        error = None
        try:
            result = zero_shot_strategy.execute(example["text"])
            example_json = {
                "intent": example["intent"],
                "priority": example["priority"],
                "route_team": example["route_team"],
                "customer_tier": example["entities"]["customer_tier"],
                "impact": example["entities"]["impact"],
                "downtime_minutes": example["entities"]["downtime_minutes"],
                "payment_risk": example["entities"]["payment_risk"],
                "has_sensitive_data": example["entities"]["has_sensitive_data"],
                "critical_feature_blocked": example["entities"]["critical_feature_blocked"],
                "language": example["entities"]["lang"],
                "action_plan_steps": example["canonical_action_plan"],
                "notify": example["notify"],
            }
            result_json = {
                "intent": result["intent"],
                "priority": result["priority"],
                "route_team": result["route_team"],
                "customer_tier": result["entities"]["customer_tier"],
                "impact": result["entities"]["impact"],
                "downtime_minutes": result["entities"]["downtime_minutes"],
                "payment_risk": result["entities"]["payment_risk"],
                "has_sensitive_data": result["entities"]["has_sensitive_data"],
                "critical_feature_blocked": result["entities"]["critical_feature_blocked"],
                "language": result["entities"]["language"],
                "action_plan_steps": result["action_plan"],
                "notify": result["notify"],
            }
        except Exception as e:
            error = str(e)
            print(f"Error en el ejemplo: {i}: {error}")
        finally:
            processing_time = time.time() - local_start_time
            print(f"[{i+1}/{len(dataset)}] Ticket procesado en {processing_time}")
            save_line_in_csv_results(
                zero_shot_csv_path,
                i,
                example["id"],
                example_json,
                result_json,
                processing_time,
                error,
                "zero_shot"
            )
    
    print(f"Tiempo de Procesamiento Total: {time.time() - start_time}")

In [22]:
zero_shot_execute(DEV_DATA[:5], "dev")

Los resultados se guardarán en: results/zero_shot_dev_20251117_080921.csv
Procesando tickets con Zero Shot...

[1/5] Processing ticket...
[1/5] Ticket procesado en 62.75812888145447

[2/5] Processing ticket...
[2/5] Ticket procesado en 90.64999151229858

[3/5] Processing ticket...
[3/5] Ticket procesado en 79.2000801563263

[4/5] Processing ticket...
[4/5] Ticket procesado en 64.12644863128662

[5/5] Processing ticket...
[5/5] Ticket procesado en 89.52449440956116
Tiempo de Procesamiento Total: 386.26254987716675


In [23]:
zero_shot_execute(TEST_PUBLIC__DATA[:5], "public")

Los resultados se guardarán en: results/zero_shot_public_20251117_081720.csv
Procesando tickets con Zero Shot...

[1/5] Processing ticket...
[1/5] Ticket procesado en 115.23297500610352

[2/5] Processing ticket...
[2/5] Ticket procesado en 71.7939178943634

[3/5] Processing ticket...
[3/5] Ticket procesado en 60.49628949165344

[4/5] Processing ticket...
[4/5] Ticket procesado en 90.96903491020203

[5/5] Processing ticket...
[5/5] Ticket procesado en 83.09526348114014
Tiempo de Procesamiento Total: 421.5908558368683


In [25]:
zero_shot_execute(TEST_HIDDEN_DATA[:5], "hidden")

Los resultados se guardarán en: results/zero_shot_hidden_20251117_082554.csv
Procesando tickets con Zero Shot...

[1/5] Processing ticket...
[1/5] Ticket procesado en 97.04732012748718

[2/5] Processing ticket...
[2/5] Ticket procesado en 103.09598135948181

[3/5] Processing ticket...
[3/5] Ticket procesado en 52.3926956653595

[4/5] Processing ticket...
[4/5] Ticket procesado en 93.05563521385193

[5/5] Processing ticket...
[5/5] Ticket procesado en 79.78936576843262
Tiempo de Procesamiento Total: 425.38433933258057


### Few Shot

In [26]:
class FewShot(BaseStrategy):
    def __init__(self, qwen: QWen):
        super().__init__(qwen)
        self.params = {
            "temperature": 0.2,
            "max_tokens": 3000,
        }

    def select_examples(self, train_data: List[Dict], n: int = 3) -> List[Dict]:
        examples = []
        intents_seen = set()

        # Priority 1: Find example with sensitive data
        for item in train_data:
            if item["entities"].get("has_sensitive_data", False):
                examples.append(item)
                intents_seen.add(item["intent"])
                break

        # Priority 2: Critical intents
        priority_intents = ["security", "outage", "cancellation"]
        for intent in priority_intents:
            if len(examples) >= n:
                break
            for item in train_data:
                if item["intent"] == intent and intent not in intents_seen:
                    examples.append(item)
                    intents_seen.add(intent)
                    break

        # Priority 3: Diverse intents
        for item in train_data:
            if len(examples) >= n:
                break
            if item["intent"] not in intents_seen:
                examples.append(item)
                intents_seen.add(item["intent"])

        return examples[:n] 

    def build_prompt(self, ticket_text: str, examples: List[Dict]) -> str:
        examples_text = ""
        for i, ex in enumerate(examples, 1):
            examples_text += f"""\nEjemplo {i}:
                Ticket: {ex['text']}\n
            """
        prompt = f"""
            Analiza estos ejemplos: {examples_text}
            Devuelve el resultado de análisis de este ticket
            Ticket:
            {ticket_text}
        """
        return prompt

    def execute(self, ticket_text: str, train_data: List[Dict]) -> Dict:
        examples = self.select_examples(train_data=train_data)
        prompt = self.build_prompt(ticket_text=ticket_text, examples=examples)
        response = self.qwen.execute(
            prompt=prompt,
            system=SYSTEM_PROMPT,
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
        )
        return clean_json_response(response)

In [27]:
few_shot_strategy = FewShot(qwen)

In [28]:
def few_shot_execute(dataset, datatype: str):
    few_shot_csv_path = init_csv_results(f"few_shot_{datatype}")

    print(f"Los resultados se guardarán en: {few_shot_csv_path}")
    train_data = TRAIN_DATA
    
    print(f"Procesando tickets con Few Shot...")
    start_time = time.time()
    for i, example in enumerate(dataset):
        print(f"\n[{i+1}/{len(dataset)}] Processing ticket...")
        local_start_time = time.time()
        error = None
        try:
            result = few_shot_strategy.execute(example["text"], train_data)
            example_json = {
                "intent": example["intent"],
                "priority": example["priority"],
                "route_team": example["route_team"],
                "customer_tier": example["entities"]["customer_tier"],
                "impact": example["entities"]["impact"],
                "downtime_minutes": example["entities"]["downtime_minutes"],
                "payment_risk": example["entities"]["payment_risk"],
                "has_sensitive_data": example["entities"]["has_sensitive_data"],
                "critical_feature_blocked": example["entities"]["critical_feature_blocked"],
                "language": example["entities"]["lang"],
                "action_plan_steps": example["canonical_action_plan"],
                "notify": example["notify"],
            }
            result_json = {
                "intent": result["intent"],
                "priority": result["priority"],
                "route_team": result["route_team"],
                "customer_tier": result["entities"]["customer_tier"],
                "impact": result["entities"]["impact"],
                "downtime_minutes": result["entities"]["downtime_minutes"],
                "payment_risk": result["entities"]["payment_risk"],
                "has_sensitive_data": result["entities"]["has_sensitive_data"],
                "critical_feature_blocked": result["entities"]["critical_feature_blocked"],
                "language": result["entities"]["language"],
                "action_plan_steps": result["action_plan"],
                "notify": result["notify"],
            }
        except Exception as e:
            error = str(e)
            print(f"Error en el ejemplo: {i}: {error}")
        finally:
            processing_time = time.time() - local_start_time
            print(f"[{i+1}/{len(dataset)}] Ticket procesado en {processing_time}")
            save_line_in_csv_results(
                few_shot_csv_path,
                i,
                example["id"],
                example_json,
                result_json,
                processing_time,
                error,
                "few_shot"
            )
    
    print(f"Tiempo de Procesamiento Total: {time.time() - start_time}")

In [29]:
few_shot_execute(DEV_DATA[:5], "dev")

Los resultados se guardarán en: results/few_shot_dev_20251117_083509.csv
Procesando tickets con Few Shot...

[1/5] Processing ticket...
[1/5] Ticket procesado en 60.46745467185974

[2/5] Processing ticket...
[2/5] Ticket procesado en 72.96562314033508

[3/5] Processing ticket...
[3/5] Ticket procesado en 64.21148657798767

[4/5] Processing ticket...
[4/5] Ticket procesado en 133.1074402332306

[5/5] Processing ticket...
[5/5] Ticket procesado en 108.27829098701477
Tiempo de Procesamiento Total: 439.0335533618927


In [30]:
few_shot_execute(TEST_PUBLIC__DATA[:5], "public")

Los resultados se guardarán en: results/few_shot_public_20251117_084324.csv
Procesando tickets con Few Shot...

[1/5] Processing ticket...
[1/5] Ticket procesado en 84.12512493133545

[2/5] Processing ticket...
[2/5] Ticket procesado en 97.1276023387909

[3/5] Processing ticket...
Error en el ejemplo: 2: No JSON object found in response
[3/5] Ticket procesado en 233.96734046936035

[4/5] Processing ticket...
[4/5] Ticket procesado en 90.43063640594482

[5/5] Processing ticket...
[5/5] Ticket procesado en 99.89935398101807
Tiempo de Procesamiento Total: 605.5532290935516


In [31]:
few_shot_execute(TEST_HIDDEN_DATA[:5], "hidden")

Los resultados se guardarán en: results/few_shot_hidden_20251117_085400.csv
Procesando tickets con Few Shot...

[1/5] Processing ticket...
[1/5] Ticket procesado en 86.74640989303589

[2/5] Processing ticket...
[2/5] Ticket procesado en 52.37698793411255

[3/5] Processing ticket...
Error en el ejemplo: 2: No JSON object found in response
[3/5] Ticket procesado en 229.579274892807

[4/5] Processing ticket...
[4/5] Ticket procesado en 74.50672245025635

[5/5] Processing ticket...
[5/5] Ticket procesado en 83.50783014297485
Tiempo de Procesamiento Total: 526.7205154895782


### CoT

In [32]:
class CoT(BaseStrategy):
    def __init__(self, qwen: QWen):
        super().__init__(qwen)
        self.params = {
            "temperature": 0.7,
            "max_tokens": 3000,
        }

    def build_prompt(self, ticket_text: str) -> str:
        prompt = f"""
            Resuelve el siguiente análisis de ticket paso a paso con razonamiento claro y ordenado.

            Pasos a seguir:
            1. IDENTIFICAR INTENT: ¿Qué tipo de problema reporta el ticket?
            (security, outage, login, billing, cancellation, bug_report, feature_request)

            2. EXTRAER ENTIDADES del ticket:
            - customer_tier: ¿Es free, standard o enterprise?
            - impact: ¿Afecta a single_user, multi_users_same_team u org_wide?
            - downtime_minutes: ¿Cuánto tiempo de inactividad reporta? (número)
            - payment_risk: ¿Hay riesgo de pago? (low, medium, high)
            - has_sensitive_data: ¿Contiene datos sensibles (PII, credenciales)? (true/false)
            - critical_feature_blocked: ¿Bloquea funcionalidad crítica? (true/false)
            - language: ¿En qué idioma está el ticket? (es/en)

            3. APLICAR REGLAS DE PRIORIDAD:
            - P1: Si intent = security, O (outage + org_wide), O downtime > 120
            - P2: Si (outage + 30-120 min), O (login + multi_users), O payment_risk = high
            - P3: Si (billing + no high risk), O (bug_report + critical), O (cancellation + enterprise)
            - P4: Todos los demás casos
            - ELEVAR: Si has_sensitive_data = true, elevar un nivel (P4→P3, P3→P2, P2→P1)

            4. DETERMINAR ROUTING según intent:
            - security → security_ir | outage → infra_oncall | login → auth_platform
            - billing/cancellation → billing_ops | bug_report → app_backend | feature_request → product_mgmt

            5. GENERAR ACTION PLAN: 3-6 pasos en el mismo idioma del ticket
            Si has_sensitive_data=true, incluir paso de sanitización

            Ejemplos:

            Ejemplo 1:
                Ticket: "CRITICAL: API down for 2+ hours, all customers affected, revenue impact"
                Paso 1: Intent = outage (caída del servicio)
                Paso 2: Entidades = customer_tier: standard, impact: org_wide, downtime_minutes: 120, payment_risk: high, has_sensitive_data: false, critical_feature_blocked: true, language: en
                Paso 3: Prioridad = P1 (outage + org_wide + downtime=120)
                Paso 4: Route = infra_oncall (outage → infra_oncall)
                Paso 5: Action plan = 3 pasos en inglés
                FINAL: {{"intent": "outage", "priority": "P1", "route_team": "infra_oncall"}}

            Ejemplo 2:
                Ticket: "Enterprise client cancelling contract due to billing issues, payment overdue"
                Paso 1: Intent = cancellation (cancelación de contrato)
                Paso 2: Entidades = customer_tier: enterprise, impact: single_user, downtime_minutes: 0, payment_risk: high, has_sensitive_data: false, critical_feature_blocked: false, language: en
                Paso 3: Prioridad = P3 (cancellation + enterprise)
                Paso 4: Route = billing_ops, Notify = cx_retention (regla especial enterprise)
                Paso 5: Action plan = 4 pasos en inglés
                FINAL: {{"intent": "cancellation", "priority": "P3", "route_team": "billing_ops", "notify": ["cx_retention"]}}

            Ejemplo 3:
                Ticket: "Usuario reporta error en login con credenciales expuestas en logs"
                Paso 1: Intent = login (problema de autenticación)
                Paso 2: Entidades = customer_tier: free, impact: single_user, downtime_minutes: 0, payment_risk: low, has_sensitive_data: true (credenciales expuestas), critical_feature_blocked: false, language: es
                Paso 3: Prioridad = P4 base → P3 (elevado por has_sensitive_data)
                Paso 4: Route = auth_platform (login → auth_platform)
                Paso 5: Action plan = 4 pasos en español con sanitización
                FINAL: {{"intent": "login", "priority": "P3", "route_team": "auth_platform"}}

            Problema:
            {ticket_text}
            """
        return prompt

    def execute(self, ticket_text: str) -> Dict:
        """Execute Chain-of-Thought analysis"""
        prompt = self.build_prompt(ticket_text=ticket_text)
        response = self.qwen.execute(
            prompt=prompt,
            system=SYSTEM_PROMPT,
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
        )
        return clean_json_response(response)

In [33]:
cot_strategy = CoT(qwen)

In [36]:
def cot_execute(dataset, datatype: str):
    cot_csv_path = init_csv_results(f"cot_{datatype}")
    print(f"Los resultados se guardarán en: {cot_csv_path}")
    
    print(f"Procesando tickets con CoT...")
    start_time = time.time()
    for i, example in enumerate(dataset):
        print(f"\n[{i+1}/{len(dataset)}] Processing ticket...")
        local_start_time = time.time()
        error = None
        try:
            result = cot_strategy.execute(example["text"])
            example_json = {
                "intent": example["intent"],
                "priority": example["priority"],
                "route_team": example["route_team"],
                "customer_tier": example["entities"]["customer_tier"],
                "impact": example["entities"]["impact"],
                "downtime_minutes": example["entities"]["downtime_minutes"],
                "payment_risk": example["entities"]["payment_risk"],
                "has_sensitive_data": example["entities"]["has_sensitive_data"],
                "critical_feature_blocked": example["entities"]["critical_feature_blocked"],
                "language": example["entities"]["lang"],
                "action_plan_steps": example["canonical_action_plan"],
                "notify": example["notify"],
            }
            result_json = {
                "intent": result["intent"],
                "priority": result["priority"],
                "route_team": result["route_team"],
                "customer_tier": result["entities"]["customer_tier"],
                "impact": result["entities"]["impact"],
                "downtime_minutes": result["entities"]["downtime_minutes"],
                "payment_risk": result["entities"]["payment_risk"],
                "has_sensitive_data": result["entities"]["has_sensitive_data"],
                "critical_feature_blocked": result["entities"]["critical_feature_blocked"],
                "language": result["entities"]["language"],
                "action_plan_steps": result["action_plan"],
                "notify": result["notify"],
            }
        except Exception as e:
            error = str(e)
            print(f"Error en el ejemplo: {i}: {error}")
        finally:
            processing_time = time.time() - local_start_time
            print(f"[{i+1}/{len(dataset)}] Ticket procesado en {processing_time}")
            save_line_in_csv_results(
                cot_csv_path,
                i,
                example["id"],
                example_json,
                result_json,
                processing_time,
                error,
                "cot"
            )
    
    print(f"Tiempo de Procesamiento Total: {time.time() - start_time}")

In [37]:
cot_execute(DEV_DATA[:5], "dev")

Los resultados se guardarán en: results/cot_dev_20251117_090659.csv
Procesando tickets con CoT...

[1/5] Processing ticket...
[1/5] Ticket procesado en 53.704527139663696

[2/5] Processing ticket...
[2/5] Ticket procesado en 86.21174669265747

[3/5] Processing ticket...
[3/5] Ticket procesado en 44.7709527015686

[4/5] Processing ticket...
[4/5] Ticket procesado en 57.19769644737244

[5/5] Processing ticket...
[5/5] Ticket procesado en 98.51042413711548
Tiempo de Procesamiento Total: 340.39872336387634


In [38]:
cot_execute(TEST_PUBLIC__DATA[:5], "public")

Los resultados se guardarán en: results/cot_public_20251117_091301.csv
Procesando tickets con CoT...

[1/5] Processing ticket...
[1/5] Ticket procesado en 156.8422679901123

[2/5] Processing ticket...
[2/5] Ticket procesado en 69.98521375656128

[3/5] Processing ticket...
[3/5] Ticket procesado en 65.6616632938385

[4/5] Processing ticket...
[4/5] Ticket procesado en 52.11466336250305

[5/5] Processing ticket...
[5/5] Ticket procesado en 43.392088174819946
Tiempo de Procesamiento Total: 387.99899077415466


In [39]:
cot_execute(TEST_HIDDEN_DATA[:5], "hidden")

Los resultados se guardarán en: results/cot_hidden_20251117_092006.csv
Procesando tickets con CoT...

[1/5] Processing ticket...
[1/5] Ticket procesado en 103.14505791664124

[2/5] Processing ticket...
[2/5] Ticket procesado en 53.02241396903992

[3/5] Processing ticket...
[3/5] Ticket procesado en 48.964380979537964

[4/5] Processing ticket...
[4/5] Ticket procesado en 45.85622596740723

[5/5] Processing ticket...
[5/5] Ticket procesado en 64.04537177085876
Tiempo de Procesamiento Total: 315.03654289245605


### CoT - SC

In [40]:
class CoTSC(CoT):
    def __init__(self, qwen: QWen):
        super().__init__(qwen)
        self.params = {
            "temperature": 0.8,
            "max_tokens": 4000,
            "trials": 3,
        }

    def _vote_on_field(self, answers: list, key: str) -> tuple:
        votes = Counter([r.get(key) for r in answers if r.get(key)])
        winner = votes.most_common(1)[0][0] if votes else None
        return votes, winner

    def _format_votes(self, votes: Counter) -> str:
        return ", ".join([f"{k}: {v}" for k, v in votes.items()])

    def execute(self, ticket_text: str) -> Dict:
        answers = []
        prompt = self.build_prompt(ticket_text=ticket_text)
        trials = self.params["trials"]

        # Generando múltiples rutas de razonamiento
        for trial in range(trials):
            print(f"Trial {trial + 1}...")     
            trial_start_time = time.time()
            response = self.qwen.execute(
                prompt=prompt,
                system=SYSTEM_PROMPT,
                temperature=self.params["temperature"],
                max_tokens=self.params["max_tokens"],
            )
            try:
                parsed = clean_json_response(response)
                timing = time.time() - trial_start_time
                parsed["timing"] = timing
                print(f"Trial timing: {timing}")
                answers.append(parsed)
            except Exception as e:
                print(f"Trial {trial+1}/{trials} falló al parsearse: {e}")
                continue

        if not answers:
            raise ValueError("Todos los intentos CoT-SC fallaron en parsearse")

        # Votación en campos "key" para buscar un consenso
        intent_votes, winner_intent = self._vote_on_field(answers, "intent")
        priority_votes, winner_priority = self._vote_on_field(answers, "priority")
        route_votes, winner_route = self._vote_on_field(answers, "route_team")

        # Selecciona la respuesta más cercana al consenso (highest match score)
        winner = None
        winner_index = -1
        best_score = -1
        for index, response in enumerate(answers):
            score = 0
            if response.get("intent") == winner_intent:
                score += 1
            if response.get("priority") == winner_priority:
                score += 1
            if response.get("route_team") == winner_route:
                score += 1
            if score > best_score:
                best_score = score
                winner = response
                winner_index = index

        return {
            "winner": winner,
            "winner_index": winner_index,
            "votes": {
                "intent": self._format_votes(intent_votes),
                "priority": self._format_votes(priority_votes),
                "route_team": self._format_votes(route_votes),
            },
            "answers": answers,
            "metadata": {
                "trials": trials,
                "consensus_score": f"{best_score}/3",
                "intent_votes": dict(intent_votes),
                "priority_votes": dict(priority_votes),
                "route_votes": dict(route_votes),
            },
        }

In [41]:
cot_sc_strategy = CoTSC(qwen)

In [44]:
def cot_sc_execute(dataset, datatype: str):
    cot_sc_csv_path = init_csv_results(f"cot_sc_{datatype}")
    print(f"Los resultados se guardarán en: {cot_sc_csv_path}")
    
    print(f"Procesando tickets con CoT SC...")
    start_time = time.time()
    for i, example in enumerate(dataset):
        print(f"\n[{i+1}/{len(dataset)}] Processing ticket...")
        local_start_time = time.time()
        error = None
        try:
            result = cot_sc_strategy.execute(example["text"])
            example_json = {
                "intent": example["intent"],
                "priority": example["priority"],
                "route_team": example["route_team"],
                "customer_tier": example["entities"]["customer_tier"],
                "impact": example["entities"]["impact"],
                "downtime_minutes": example["entities"]["downtime_minutes"],
                "payment_risk": example["entities"]["payment_risk"],
                "has_sensitive_data": example["entities"]["has_sensitive_data"],
                "critical_feature_blocked": example["entities"]["critical_feature_blocked"],
                "language": example["entities"]["lang"],
                "action_plan_steps": example["canonical_action_plan"],
                "notify": example["notify"],
            }
            result_json = {
                "intent": result["winner"]["intent"],
                "priority": result["winner"]["priority"],
                "route_team": result["winner"]["route_team"],
                "customer_tier": result["winner"]["entities"]["customer_tier"],
                "impact": result["winner"]["entities"]["impact"],
                "downtime_minutes": result["winner"]["entities"]["downtime_minutes"],
                "payment_risk": result["winner"]["entities"]["payment_risk"],
                "has_sensitive_data": result["winner"]["entities"]["has_sensitive_data"],
                "critical_feature_blocked": result["winner"]["entities"]["critical_feature_blocked"],
                "language": result["winner"]["entities"]["language"],
                "action_plan_steps": result["winner"]["action_plan"],
                "notify": result["winner"]["notify"],

                "trials": result["metadata"]["trials"],
                "trial_winner": result["winner_index"],
            }
        except Exception as e:
            error = str(e)
            print(f"Error en el ejemplo: {i}: {error}")
        finally:
            processing_time = time.time() - local_start_time
            print(f"[{i+1}/{len(dataset)}] Ticket procesado en {processing_time}")
            save_line_in_csv_results(
                cot_sc_csv_path,
                i,
                example["id"],
                example_json,
                result_json,
                processing_time,
                error,
                "cot_sc"
            )
    
    print(f"Tiempo de Procesamiento Total: {time.time() - start_time}")

In [45]:
cot_sc_execute(DEV_DATA[:5], "dev")

Los resultados se guardarán en: results/cot_sc_dev_20251117_093825.csv
Procesando tickets con CoT SC...

[1/5] Processing ticket...
Trial 1...
Trial timing: 49.89475893974304
Trial 2...
Trial timing: 55.43874764442444
Trial 3...
Trial timing: 37.53117799758911
[1/5] Ticket procesado en 142.865159034729

[2/5] Processing ticket...
Trial 1...
Trial timing: 51.12953567504883
Trial 2...
Trial timing: 56.4077365398407
Trial 3...
Trial timing: 58.61653661727905
[2/5] Ticket procesado en 166.15426635742188

[3/5] Processing ticket...
Trial 1...
Trial timing: 41.17063546180725
Trial 2...
Trial timing: 38.00737643241882
Trial 3...
Trial timing: 38.6243896484375
[3/5] Ticket procesado en 117.80286359786987

[4/5] Processing ticket...
Trial 1...
Trial timing: 107.88783407211304
Trial 2...
Trial timing: 141.51513528823853
Trial 3...
Trial timing: 48.71985459327698
[4/5] Ticket procesado en 298.1232888698578

[5/5] Processing ticket...
Trial 1...
Trial timing: 134.32797527313232
Trial 2...
Trial ti

In [46]:
cot_sc_execute(TEST_PUBLIC__DATA[:5], "public")

Los resultados se guardarán en: results/cot_sc_public_20251117_095716.csv
Procesando tickets con CoT SC...

[1/5] Processing ticket...
Trial 1...
Trial timing: 55.67978310585022
Trial 2...
Trial timing: 139.31627202033997
Trial 3...
Trial timing: 45.511396169662476
[1/5] Ticket procesado en 240.50794053077698

[2/5] Processing ticket...
Trial 1...
Trial timing: 72.94678568840027
Trial 2...
Trial timing: 58.06888937950134
Trial 3...
Trial timing: 103.81067943572998
[2/5] Ticket procesado en 234.82684993743896

[3/5] Processing ticket...
Trial 1...
Trial timing: 40.59628653526306
Trial 2...
Trial timing: 37.394222259521484
Trial 3...
Trial timing: 37.708237409591675
[3/5] Ticket procesado en 115.69920682907104

[4/5] Processing ticket...
Trial 1...
Trial timing: 38.58621406555176
Trial 2...
Trial timing: 101.59349870681763
Trial 3...
Trial timing: 73.79157018661499
[4/5] Ticket procesado en 213.97173762321472

[5/5] Processing ticket...
Trial 1...
Trial timing: 85.89443850517273
Trial 2.

In [47]:
cot_sc_execute(TEST_HIDDEN_DATA[:5], "hidden")

Los resultados se guardarán en: results/cot_sc_hidden_20251117_101445.csv
Procesando tickets con CoT SC...

[1/5] Processing ticket...
Trial 1...
Trial timing: 85.79219841957092
Trial 2...
Trial timing: 94.9577317237854
Trial 3...
Trial timing: 172.89947366714478
[1/5] Ticket procesado en 353.6498899459839

[2/5] Processing ticket...
Trial 1...
Trial timing: 52.62861704826355
Trial 2...
Trial timing: 55.35444641113281
Trial 3...
Trial timing: 79.67632055282593
[2/5] Ticket procesado en 187.65984892845154

[3/5] Processing ticket...
Trial 1...
Trial timing: 50.073885679244995
Trial 2...
Trial timing: 53.033077239990234
Trial 3...
Trial timing: 41.66406989097595
[3/5] Ticket procesado en 144.77149748802185

[4/5] Processing ticket...
Trial 1...
Trial timing: 57.3955819606781
Trial 2...
Trial timing: 102.3822922706604
Trial 3...
Trial timing: 99.12914323806763
[4/5] Ticket procesado en 258.9075171947479

[5/5] Processing ticket...
Trial 1...
Trial timing: 44.84618067741394
Trial 2...
Tria

### ToT

In [48]:
@dataclass
class TotNode:
    ticket_text: str
    state: Dict = field(default_factory=dict)  # Estado actual del análisis
    path: List[str] = field(default_factory=list)  # Acciones tomadas para alcanzar el nodo
    score: float = 0.0
    note: str = ""  # Razonamiento LLM reasoning para el score

In [49]:
class ToT(BaseStrategy):
    def __init__(self, qwen: QWen):
        super().__init__(qwen)
        self.params = {
            "temperature": 0.3,
            "max_tokens": 5000,
            "beam_search": 2,
            "depth": 4,
        }

    def _initial_state(self, ticket_text: str) -> Dict:
        return {
            "ticket_text": ticket_text,
            "intent": None,
            "entities": None,
            "priority": None,
            "route_team": None,
            "notify": None,
            "action_plan": None,
            "stage": "init",  # init -> intent_exploration -> entity_extraction -> complete
        }

    def _goal(self, state: Dict) -> bool:
        return (
            state.get("stage") == "complete"
            and state.get("intent") is not None
            and state.get("priority") is not None
            and state.get("action_plan") is not None
        )

    def _score_state(self, node: TotNode) -> Tuple[float, str]:
        state = node.state
        stage = state.get("stage", "init")

        # Build resumen status para el LLM
        status_parts = []
        status_parts.append(f"Etapa: {stage}")
        if state.get("intent"):
            status_parts.append(f"Intent: {state['intent']}")
        if state.get("entities"):
            status_parts.append(f"Entidades: extraídas")
        if state.get("priority"):
            status_parts.append(f"Prioridad: {state['priority']}")
        if state.get("action_plan"):
            status_parts.append(f"Plan: {len(state['action_plan'])} pasos")

        status = " | ".join(status_parts)
        path_desc = " → ".join(node.path) if node.path else "inicio"

        # Scoring basado en LLM
        prompt = f"""
            Estamos analizando un ticket de soporte usando Tree-of-Thoughts.

            Ticket (primeros 150 caracteres):
            {node.ticket_text[:150]}...

            Estado actual del análisis:
            {status}

            Camino tomado: {path_desc}

            Meta: Análisis completo con intent, priority, route_team, entities y action_plan válidos.

            Evalúa el progreso hacia la meta con un puntaje de 0 a 10:
            - 0-2: Muy poco progreso, estado inicial
            - 3-5: Progreso moderado, algunas decisiones tomadas
            - 6-8: Buen progreso, falta poco para completar
            - 9-10: Análisis completo o casi completo

            Primero razona brevemente (1-2 frases) y luego entrega SOLO:
            SCORE: <número entre 0 y 10>
        """
        response = self.qwen.execute(
            prompt=prompt.strip(),
            system="Eres un evaluador riguroso y conciso.",
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
            top_p=0.95,
        )

        # Extracción de score
        m = re.search(r"SCORE:\s*([0-9]+(?:\.[0-9]+)?)", response, re.IGNORECASE)
        score = float(m.group(1)) if m else 0.0

        return score, response.strip()

    def _successors(self, node: TotNode) -> List[Tuple[str, Dict]]:
        state = node.state
        stage = state.get("stage", "init")
        successors = []

        if stage == "init":
            # Step 1: Genera top-3 candidatos intent
            prompt = f"""
                Ticket: {node.ticket_text}

                Identifica los 3 intents más probables de esta lista:
                security, outage, login, billing, cancellation, bug_report, feature_request

                Responde SOLO con formato:
                CANDIDATES: intent1, intent2, intent3
            """
            response = self.qwen.execute(
                prompt=prompt.strip(),
                system=SYSTEM_PROMPT,
                temperature=self.params["temperature"],
                max_tokens=self.params["max_tokens"],
            )

            # Extracción de candidatos
            candidates = []
            m = re.search(r"CANDIDATES:\s*(.+)", response, re.IGNORECASE)
            if m:
                candidates = [c.strip() for c in m.group(1).split(",")]

            if not candidates:
                candidates = ["bug_report", "feature_request", "outage"]

            # Create successor for each candidate
            for candidate in candidates[:3]:
                action = f"Explorar intent: {candidate}"
                new_state = state.copy()
                new_state["intent"] = candidate
                new_state["stage"] = "intent_exploration"
                successors.append((action, new_state))

        elif stage == "intent_exploration":
            # Step 2: Extracción de entities para intent seleccionado
            prompt = f"""
                Ticket: {node.ticket_text}

                Intent seleccionado: {state['intent']}

                Extrae las entidades según las reglas. Devuelve JSON con:
                {{
                    "customer_tier": "free|standard|enterprise",
                    "impact": "single_user|multi_users_same_team|org_wide",
                    "downtime_minutes": 0,
                    "payment_risk": "low|medium|high",
                    "has_sensitive_data": false,
                    "critical_feature_blocked": false,
                    "language": "es|en"
                }}
            """
            response = self.qwen.execute(
                prompt=prompt.strip(),
                system=SYSTEM_PROMPT,
                temperature=self.params["temperature"],
                max_tokens=self.params["max_tokens"],
            )

            try:
                entities = clean_json_response(response)
                action = "Extraer entidades"
                new_state = state.copy()
                new_state["entities"] = entities
                new_state["stage"] = "entity_extraction"
                successors.append((action, new_state))
            except Exception as e:
                print(f"[ToT] Failed to extract entities: {e}")
                # Fallback with default entities
                default_entities = {
                    "customer_tier": "standard",
                    "impact": "single_user",
                    "downtime_minutes": 0,
                    "payment_risk": "low",
                    "has_sensitive_data": False,
                    "critical_feature_blocked": False,
                    "language": "es",
                }
                action = "Extraer entidades (con defaults)"
                new_state = state.copy()
                new_state["entities"] = default_entities
                new_state["stage"] = "entity_extraction"
                successors.append((action, new_state))

        elif stage == "entity_extraction":
            # Step 3: Genera análisis completo
            prompt = f"""
                Devuelve el resultado de análisis de este ticket
                Ticket:
                {node.ticket_text}
            """
            response = self.qwen.execute(
                prompt=prompt,
                system=SYSTEM_PROMPT,
                temperature=self.params["temperature"],
                max_tokens=self.params["max_tokens"],
            )

            try:
                result = clean_json_response(response)
                # Sobreescribe con valores ToT explorados
                result["intent"] = state["intent"]
                result["entities"] = state["entities"]

                action = "Completar análisis"
                new_state = state.copy()
                new_state.update(result)
                new_state["stage"] = "complete"
                successors.append((action, new_state))
            except Exception as e:
                print(f"[ToT] Failed to complete analysis: {e}")
                # Fallback con resultado mínimo válido
                action = "Completar análisis (fallback)"
                new_state = state.copy()
                new_state["priority"] = "P3"
                new_state["route_team"] = self.calculate_expected_route(state["intent"])
                new_state["notify"] = []
                new_state["action_plan"] = [
                    "Analizar el ticket en detalle",
                    "Contactar al usuario para más información",
                    "Escalar según sea necesario",
                ]
                new_state["stage"] = "complete"
                successors.append((action, new_state))

        return successors

    def execute(self, ticket_text: str) -> Dict:
        b = self.params["beam_search"]
        d = self.params["depth"]
        
        # Inicializa con el nodo root
        start_state = self._initial_state(ticket_text)
        best = TotNode(ticket_text, start_state, [], 0.0, "inicio")
        frontier = [best]  # La frontera son las opciones actuales

        for depth in range(d):
            print(f"[ToT] Depth {depth}: Frontier size = {len(frontier)}")

            # Checkeo para el estado meta en frontier (early termination)
            for node in frontier:
                if self._goal(node.state):
                    print(f"[ToT] Goal reached at depth {depth}!")
                    result = node.state.copy()
                    # Remove internal tracking fields
                    result.pop("ticket_text", None)
                    result.pop("stage", None)
                    result["_tot_metadata"] = {
                        "beam_width": b,
                        "final_depth": depth,
                        "final_score": node.score,
                        "reasoning_path": node.path,
                        # "note": node.note,
                    }
                    return result

            # Generación de todos los candidatos sucesores
            candidates = []
            for node in frontier:
                for action, s2 in self._successors(node):
                    new_path = node.path + [action]
                    new_node = TotNode(ticket_text, s2, new_path, 0.0, "")
                    sc, note = self._score_state(new_node)
                    new_node.score = sc
                    new_node.note = note
                    candidates.append(new_node)

            if not candidates:
                print(f"[ToT] No candidates at depth {depth}, stopping")
                break

            # Mantiene el top-b candidatos (beam search)
            candidates.sort(key=lambda n: n.score, reverse=True)
            frontier = candidates[:b]
            best = frontier[0]
            print(f"[ToT] Best: stage={best.state.get('stage')}, score={best.score:.2f}")

            # Checkea si el mejor nodo alcanzó la meta
            if self._goal(best.state):
                print(f"[ToT] Goal reached with best node at depth {depth}!")
                result = best.state.copy()
                result.pop("ticket_text", None)
                result.pop("stage", None)
                result["_tot_metadata"] = {
                    "beam_width": b,
                    "final_depth": depth,
                    "final_score": best.score,
                    "reasoning_path": best.path,
                    # "note": best.note,
                }
                return result

        # Máx. profunidad alcanzada (depth) - devuelve el mejor noo encontrado
        print(f"[ToT] Máx. Depth alcanzado. Mejor Nodo: stage={best.state.get('stage')}, score={best.score:.2f}")
        result = best.state.copy()
        result.pop("ticket_text", None)
        result.pop("stage", None)

        # Asegura tener el mínimo de campos requeridos
        if not result.get("intent"):
            print("[ToT] intent No encontrado, falling back -> zero_shot")
            result = zero_shot.execute(ticket_text)
            result["_tot_metadata"] = {
                "beam_width": b,
                "final_depth": depth,
                "final_score": 0.0,
                "reasoning_path": [],
                "fallback": "zero_shot",
            }
        else:
            result["_tot_metadata"] = {
                "beam_width": b,
                "final_depth": depth,
                "final_score": best.score,
                "reasoning_path": best.path,
                # "note": best.note,
                "incomplete": not self._goal(best.state),
            }

        return result

In [50]:
tot_strategy = ToT(qwen)

In [61]:
def tot_execute(dataset, datatype: str):
    tot_csv_path = init_csv_results(f"tot_{datatype}")
    print(f"Los resultados se guardarán en: {tot_csv_path}")

    print(f"Procesando tickets con ToT...")
    start_time = time.time()
    for i, example in enumerate(dataset):
        print(f"\n[{i+1}/{len(dataset)}] Processing ticket...")
        local_start_time = time.time()
        error = None
        try:
            result = tot_strategy.execute(example["text"])
            example_json = {
                "intent": example["intent"],
                "priority": example["priority"],
                "route_team": example["route_team"],
                "customer_tier": example["entities"]["customer_tier"],
                "impact": example["entities"]["impact"],
                "downtime_minutes": example["entities"]["downtime_minutes"],
                "payment_risk": example["entities"]["payment_risk"],
                "has_sensitive_data": example["entities"]["has_sensitive_data"],
                "critical_feature_blocked": example["entities"]["critical_feature_blocked"],
                "language": example["entities"]["lang"],
                "action_plan_steps": example["canonical_action_plan"],
                "notify": example["notify"],
            }
            result_json = {
                "intent": result["intent"],
                "priority": result["priority"],
                "route_team": result["route_team"],
                "customer_tier": result["entities"].get("customer_tier", ""),
                "impact": result["entities"].get("impact", ""),
                "downtime_minutes": result["entities"].get("downtime_minutes", -1),
                "payment_risk": result["entities"].get("payment_risk", ""),
                "has_sensitive_data": result["entities"].get("has_sensitive_data", None),
                "critical_feature_blocked": result["entities"].get("critical_feature_blocked", None),
                "language": result["entities"].get("language", ""),
                "action_plan_steps": result["action_plan"],
                "notify": result["notify"],
    
                "beam_width": result["_tot_metadata"]["beam_width"],
                "final_depth": result["_tot_metadata"]["final_depth"],
                "final_score": result["_tot_metadata"]["final_score"],
            }
        except Exception as e:
            error = str(e)
            print(f"Error en el ejemplo: {i}: {error}")
            print("ERROR COMPLETO:")
            traceback.print_exc()
        finally:
            processing_time = time.time() - local_start_time
            print(f"[{i+1}/{len(dataset)}] Ticket procesado en {processing_time}")
            save_line_in_csv_results(
                tot_csv_path,
                i,
                example["id"],
                example_json,
                result_json,
                processing_time,
                error,
                "tot"
            )
    
    print(f"Tiempo de Procesamiento Total: {time.time() - start_time}")

In [62]:
tot_execute(DEV_DATA[:5], "dev")

Los resultados se guardarán en: results/tot_dev_20251117_143723.csv
Procesando tickets con ToT...

[1/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=7.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[ToT] Depth 2: Frontier size = 2
[ToT] Best: stage=complete, score=9.00
[ToT] Goal reached with best node at depth 2!
[1/5] Ticket procesado en 491.32092666625977

[2/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=7.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[ToT] Depth 2: Frontier size = 2
[ToT] Best: stage=complete, score=9.00
[ToT] Goal reached with best node at depth 2!
[2/5] Ticket procesado en 489.5107328891754

[3/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=7.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[ToT

In [63]:
tot_execute(TEST_PUBLIC__DATA[:5], "public")

Los resultados se guardarán en: results/tot_public_20251117_153654.csv
Procesando tickets con ToT...

[1/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=7.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[ToT] Depth 2: Frontier size = 2
[ToT] Best: stage=complete, score=8.00
[ToT] Goal reached with best node at depth 2!
[1/5] Ticket procesado en 501.26083183288574

[2/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=6.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[ToT] Depth 2: Frontier size = 2
[ToT] Best: stage=complete, score=9.00
[ToT] Goal reached with best node at depth 2!
[2/5] Ticket procesado en 449.0557379722595

[3/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=7.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[

In [64]:
tot_execute(TEST_HIDDEN_DATA[:5], "hidden")

Los resultados se guardarán en: results/tot_hidden_20251117_162410.csv
Procesando tickets con ToT...

[1/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=7.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[ToT] Depth 2: Frontier size = 2
[ToT] Best: stage=complete, score=8.00
[ToT] Goal reached with best node at depth 2!
[1/5] Ticket procesado en 719.0311210155487

[2/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=7.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[ToT] Depth 2: Frontier size = 2
[ToT] Best: stage=complete, score=9.00
[ToT] Goal reached with best node at depth 2!
[2/5] Ticket procesado en 513.6160237789154

[3/5] Processing ticket...
[ToT] Depth 0: Frontier size = 1
[ToT] Best: stage=intent_exploration, score=6.00
[ToT] Depth 1: Frontier size = 2
[ToT] Best: stage=entity_extraction, score=7.00
[T

### GoT

In [65]:
@dataclass
class GotNode:
    node_id: int
    name: str
    dependencies: List[int] = field(default_factory=list)  # Parent node IDs
    output: Optional[Dict] = None  # Node's computed output
    executed: bool = False

In [66]:
class GoT(BaseStrategy):
    def __init__(self, qwen):
        super().__init__(qwen)
        self.params = {
            "temperature": 0.3,
            "max_tokens": 5000,
        }

    def _node1_analyze_text(self, ticket_text: str) -> Dict:
        """
        NODO 1: Análisis de texto - Detectar idioma, keywords, contexto
        """
        prompt = f"""
            Analiza el siguiente ticket y extrae:
            1. Idioma (es o en)
            2. Palabras clave (5-10 palabras importantes)
            3. Contexto general (1 frase describiendo el problema)

            Ticket:
            {ticket_text}

            Devuelve SOLO JSON:
            {{
                "idioma": "es",
                "keywords": ["palabra1", "palabra2", ...],
                "contexto": "Descripción breve del problema"
            }}
        """
        response = self.qwen.execute(
            prompt=prompt.strip(),
            system="Eres un experto en análisis de texto. Retorna SOLO JSON válido.",
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
        )
        return clean_json_response(response)

    def _node2_identify_intent(self, node1_output: Dict, ticket_text: str) -> Dict:
        """
        NODO 2: Identificar intent basado en keywords y contexto del NODO 1
        """
        keywords = node1_output.get("keywords", [])
        contexto = node1_output.get("contexto", "")

        prompt = f"""
            Basándote en las keywords y contexto extraídos del ticket, identifica el intent.

            Keywords: {keywords}
            Contexto: {contexto}

            Ticket original: {ticket_text[:200]}...

            Intents posibles: security, outage, login, billing, cancellation, bug_report, feature_request

            Devuelve SOLO JSON:
            {{
                "intent": "outage",
                "confianza": 95
            }}
        """
        response = self.qwen.execute(
            prompt=prompt.strip(),
            system="Eres un experto en clasificación de tickets. Retorna SOLO JSON válido.",
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
        )
        return clean_json_response(response)

    def _node3_extract_entities(self, node1_output: Dict, node2_output: Dict, ticket_text: str) -> Dict:
        """
        NODO 3: Extraer entidades guiado por intent del NODO 2
        """
        intent = node2_output.get("intent", "")
        contexto = node1_output.get("contexto", "")

        prompt = f"""
            Ticket: {ticket_text}

            Contexto: {contexto}
            Intent identificado: {intent}

            Extrae las siguientes entidades del ticket:

            Devuelve SOLO JSON:
            {{
                "customer_tier": "free|standard|enterprise",
                "impact": "single_user|multi_users_same_team|org_wide",
                "downtime_minutes": 0,
                "payment_risk": "low|medium|high",
                "has_sensitive_data": false,
                "critical_feature_blocked": false,
                "language": "es|en"
            }}
        """
        response = self.qwen.execute(
            prompt=prompt.strip(),
            system=SYSTEM_PROMPT,
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
        )
        return clean_json_response(response)

    def _node4_validate_consistency(self, node2_output: Dict, node3_output: Dict) -> Dict:
        """
        NODO 4: Validar consistencia entre intent y entities (CRÍTICO PARA GoT)
        """
        intent = node2_output.get("intent", "")
        entities = node3_output

        prompt = f"""
            Valida si el intent es consistente con las entidades extraídas.

            Intent: {intent}
            Entities: {json.dumps(entities, indent=2)}

            Reglas de consistencia:
            - Si intent="outage", downtime_minutes debe ser > 0
            - Si intent="login", impact debería incluir problemas de autenticación
            - Si intent="security", has_sensitive_data podría ser true
            - Si intent="billing", payment_risk debería ser relevante

            Devuelve SOLO JSON:
            {{
                "es_consistente": true/false,
                "problemas": ["lista de inconsistencias encontradas"] o [],
                "ajustes_recomendados": {{
                    "intent": "nuevo_intent" (si debe cambiar),
                    "entities": {{"campo": "nuevo_valor"}} (si debe cambiar)
                }}
            }}
        """
        response = self.qwen.execute(
            prompt=prompt.strip(),
            system="Eres un validador riguroso de consistencia. Retorna SOLO JSON válido.",
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
        )
        return clean_json_response(response)

    def _node5_calculate_priority(self, node2_output: Dict, node3_output: Dict, node4_output: Dict) -> Dict:
        """
        NODO 5: Calcular prioridad usando reglas de negocio
        """
        intent = node2_output.get("intent", "")
        entities = node3_output
        es_consistente = node4_output.get("es_consistente", True)

        if not es_consistente:
            ajustes = node4_output.get("ajustes_recomendados", {})
            if ajustes.get("intent"):
                intent = ajustes["intent"]
            if ajustes.get("entities"):
                entities.update(ajustes["entities"])

        priority = self.calculate_expected_priority(
            intent,
            entities.get("impact", ""),
            entities.get("downtime_minutes", 0),
            entities.get("payment_risk", ""),
            entities.get("critical_feature_blocked", False),
            entities.get("customer_tier", ""),
            entities.get("has_sensitive_data", False),
        )

        return {"priority": priority}

    def _node6_determine_routing(self, node2_output: Dict, node3_output: Dict) -> Dict:
        """
        NODO 6: Determinar routing y notificaciones
        """
        intent = node2_output.get("intent", "")
        customer_tier = node3_output.get("customer_tier", "")

        route_team = self.calculate_expected_route(intent)
        notify = self.calculate_expected_notify(intent, customer_tier)

        return {"route_team": route_team, "notify": notify}

    def _node7_generate_action_plan(self, node1_output: Dict, node2_output: Dict, node3_output: Dict, node5_output: Dict, ticket_text: str) -> Dict:
        """
        NODO 7: Generar action plan en el idioma correcto
        """
        idioma = node1_output.get("idioma", "es")
        intent = node2_output.get("intent", "")
        priority = node5_output.get("priority", "P3")
        has_sensitive_data = node3_output.get("has_sensitive_data", False)

        prompt = f"""
            Genera un action plan (3-6 pasos) para este ticket.

            Ticket: {ticket_text[:200]}...
            Intent: {intent}
            Prioridad: {priority}
            Has sensitive data: {has_sensitive_data}

            IMPORTANTE:
            - Genera el plan en {"español" if idioma == "es" else "inglés"}
            - Si has_sensitive_data=true, incluye paso de sanitización
            - Adapta urgencia al nivel de prioridad {priority}

            Devuelve SOLO JSON:
            {{
                "action_plan": [
                    "Paso 1...",
                    "Paso 2...",
                    "Paso 3..."
                ]
            }}
        """
        response = self.qwen.execute(
            prompt=prompt.strip(),
            system=SYSTEM_PROMPT,
            temperature=self.params["temperature"],
            max_tokens=self.params["max_tokens"],
        )
        return clean_json_response(response)

    def execute(self, ticket_text: str, enable_backtracking: bool = True) -> Dict:
        print("[GoT] Starting Graph-of-Thoughts execution...")

        # Initialización de nodos
        nodes = {
            1: GotNode(1, "Análisis de Texto", []),
            2: GotNode(2, "Identificación de Intent", [1]),
            3: GotNode(3, "Extracción de Entidades", [1, 2]),
            4: GotNode(4, "Validación de Consistencia", [2, 3]),
            5: GotNode(5, "Cálculo de Prioridad", [2, 3, 4]),
            6: GotNode(6, "Determinación de Routing", [2, 3]),
            7: GotNode(7, "Generación de Action Plan", [1, 2, 3, 5]),
        }

        execution_log = []

        # Ejecutar nodos en orden topológico

        # NODO 1: Análisis de Texto
        print("[GoT] Executing Node 1: Análisis de Texto")
        nodes[1].output = self._node1_analyze_text(
            ticket_text
        )
        nodes[1].executed = True
        execution_log.append(f"Node 1: {nodes[1].output}")

        # NODO 2: Intent
        print("[GoT] Executing Node 2: Identificación de Intent")
        nodes[2].output = self._node2_identify_intent(
            nodes[1].output,
            ticket_text
        )
        nodes[2].executed = True
        execution_log.append(f"Node 2: {nodes[2].output}")

        # NODO 3: Entities
        print("[GoT] Executing Node 3: Extracción de Entidades")
        nodes[3].output = self._node3_extract_entities(
            nodes[1].output,
            nodes[2].output,
            ticket_text
        )
        nodes[3].executed = True
        execution_log.append(f"Node 3: {nodes[3].output}")

        # NODO 4: Validación
        print("[GoT] Executing Node 4: Validación de Consistencia")
        nodes[4].output = self._node4_validate_consistency(
            nodes[2].output,
            nodes[3].output
        )
        nodes[4].executed = True
        execution_log.append(f"Node 4: {nodes[4].output}")

        # Backtracking si hay inconsistencia
        backtracking_applied = False
        if enable_backtracking and not nodes[4].output.get("es_consistente", True):
            print("[GoT] ⚠️ Inconsistency detected! Applying backtracking adjustments...")
            ajustes = nodes[4].output.get("ajustes_recomendados", {})

            # Aplicar ajustes al intent (NODO 2)
            if ajustes.get("intent"):
                print(f"[GoT]   Adjusting intent: {nodes[2].output.get('intent')} → {ajustes['intent']}")
                nodes[2].output["intent"] = ajustes["intent"]
                backtracking_applied = True

            # Aplicar ajustes a entities (NODO 3)
            if ajustes.get("entities"):
                print(f"[GoT]   Adjusting entities: {ajustes['entities']}")
                nodes[3].output.update(ajustes["entities"])
                backtracking_applied = True

            execution_log.append(f"Backtracking applied: {nodes[4].output.get('problemas', [])}")

        # NODO 5: Priority
        print("[GoT] Executing Node 5: Cálculo de Prioridad")
        nodes[5].output = self._node5_calculate_priority(
            nodes[2].output,
            nodes[3].output,
            nodes[4].output
        )
        nodes[5].executed = True
        execution_log.append(f"Node 5: {nodes[5].output}")

        # NODO 6: Routing
        print("[GoT] Executing Node 6: Determinación de Routing")
        nodes[6].output = self._node6_determine_routing(
            nodes[2].output,
            nodes[3].output
        )
        nodes[6].executed = True
        execution_log.append(f"Node 6: {nodes[6].output}")

        # NODO 7: Action Plan
        print("[GoT] Executing Node 7: Generación de Action Plan")
        nodes[7].output = self._node7_generate_action_plan(
            nodes[1].output,
            nodes[2].output,
            nodes[3].output,
            nodes[5].output,
            ticket_text,
        )
        nodes[7].executed = True
        execution_log.append(f"Node 7: {nodes[7].output}")

        result = {
            "intent": nodes[2].output.get("intent"),
            "priority": nodes[5].output.get("priority"),
            "route_team": nodes[6].output.get("route_team"),
            "notify": nodes[6].output.get("notify", []),
            "entities": nodes[3].output,
            "action_plan": nodes[7].output.get("action_plan", []),
            "_got_metadata": {
                "method": "graph_of_thoughts_algorithmic",
                "nodes_executed": 7,
                "backtracking_applied": backtracking_applied,
                "consistency_check": nodes[4].output,
                "execution_log": execution_log,
                "graph_structure": {
                    "node_1": {"name": nodes[1].name, "dependencies": []},
                    "node_2": {"name": nodes[2].name, "dependencies": [1]},
                    "node_3": {"name": nodes[3].name, "dependencies": [1, 2]},
                    "node_4": {"name": nodes[4].name, "dependencies": [2, 3]},
                    "node_5": {"name": nodes[5].name, "dependencies": [2, 3, 4]},
                    "node_6": {"name": nodes[6].name, "dependencies": [2, 3]},
                    "node_7": {"name": nodes[7].name, "dependencies": [1, 2, 3, 5]},
                },
            },
        }

        print(f"[GoT] ✓ Ejecución de Grafos completa. Backtracking: {'Sí' if backtracking_applied else 'No'}")
        return result

In [67]:
got_strategy = GoT(qwen)

In [72]:
def got_execute(dataset, datatype: str):
    got_csv_path = init_csv_results(f"got_{datatype}")
    print(f"Los resultados se guardarán en: {got_csv_path}")

    print(f"Procesando tickets con GoT...")
    start_time = time.time()
    for i, example in enumerate(dataset):
        print(f"\n[{i+1}/{len(dataset)}] Processing ticket...")
        local_start_time = time.time()
        error = None
        try:
            result = got_strategy.execute(example["text"])
            example_json = {
                "intent": example["intent"],
                "priority": example["priority"],
                "route_team": example["route_team"],
                "customer_tier": example["entities"]["customer_tier"],
                "impact": example["entities"]["impact"],
                "downtime_minutes": example["entities"]["downtime_minutes"],
                "payment_risk": example["entities"]["payment_risk"],
                "has_sensitive_data": example["entities"]["has_sensitive_data"],
                "critical_feature_blocked": example["entities"]["critical_feature_blocked"],
                "language": example["entities"]["lang"],
                "action_plan_steps": example["canonical_action_plan"],
                "notify": example["notify"],
            }
            result_json = {
                "intent": result["intent"],
                "priority": result["priority"],
                "route_team": result["route_team"],
                "customer_tier": result["entities"].get("customer_tier", ""),
                "impact": result["entities"].get("impact", ""),
                "downtime_minutes": result["entities"].get("downtime_minutes", -1),
                "payment_risk": result["entities"].get("payment_risk", ""),
                "has_sensitive_data": result["entities"].get("has_sensitive_data", None),
                "critical_feature_blocked": result["entities"].get("critical_feature_blocked", None),
                "language": result["entities"].get("language", ""),
                "action_plan_steps": result["action_plan"],
                "notify": result["notify"],
    
                "nodes_executed": result["_got_metadata"]["nodes_executed"],
                "backtracking_applied": result["_got_metadata"]["backtracking_applied"],
                "consistency_check": result["_got_metadata"]["consistency_check"],
                "execution_log": result["_got_metadata"]["execution_log"],
            }
        except Exception as e:
            error = str(e)
            print(f"Error en el ejemplo: {i}: {error}")
            print("ERROR COMPLETO:")
            traceback.print_exc()
        finally:
            processing_time = time.time() - local_start_time
            print(f"[{i+1}/{len(dataset)}] Ticket procesado en {processing_time}")
            save_line_in_csv_results(
                got_csv_path,
                i,
                example["id"],
                example_json,
                result_json,
                processing_time,
                error,
                "got"
            )
    
    print(f"Tiempo de Procesamiento Total: {time.time() - start_time}")

In [73]:
got_execute(DEV_DATA[:5], "dev")

Los resultados se guardarán en: results/got_dev_20251117_172307.csv
Procesando tickets con GoT...

[1/5] Processing ticket...
[GoT] Starting Graph-of-Thoughts execution...
[GoT] Executing Node 1: Análisis de Texto
[GoT] Executing Node 2: Identificación de Intent
[GoT] Executing Node 3: Extracción de Entidades
[GoT] Executing Node 4: Validación de Consistencia
[GoT] Executing Node 5: Cálculo de Prioridad
[GoT] Executing Node 6: Determinación de Routing
[GoT] Executing Node 7: Generación de Action Plan
[GoT] ✓ Ejecución de Grafos completa. Backtracking: No
[1/5] Ticket procesado en 216.63152074813843

[2/5] Processing ticket...
[GoT] Starting Graph-of-Thoughts execution...
[GoT] Executing Node 1: Análisis de Texto
[GoT] Executing Node 2: Identificación de Intent
[GoT] Executing Node 3: Extracción de Entidades
[GoT] Executing Node 4: Validación de Consistencia
[GoT] ⚠️ Inconsistency detected! Applying backtracking adjustments...
[GoT]   Adjusting entities: {'impact': 'authentication_failu

In [74]:
got_execute(TEST_PUBLIC__DATA[:5], "public")

Los resultados se guardarán en: results/got_public_20251117_174845.csv
Procesando tickets con GoT...

[1/5] Processing ticket...
[GoT] Starting Graph-of-Thoughts execution...
[GoT] Executing Node 1: Análisis de Texto
[GoT] Executing Node 2: Identificación de Intent
[GoT] Executing Node 3: Extracción de Entidades
[GoT] Executing Node 4: Validación de Consistencia
[GoT] Executing Node 5: Cálculo de Prioridad
[GoT] Executing Node 6: Determinación de Routing
[GoT] Executing Node 7: Generación de Action Plan
[GoT] ✓ Ejecución de Grafos completa. Backtracking: No
[1/5] Ticket procesado en 268.8183419704437

[2/5] Processing ticket...
[GoT] Starting Graph-of-Thoughts execution...
[GoT] Executing Node 1: Análisis de Texto
[GoT] Executing Node 2: Identificación de Intent
[GoT] Executing Node 3: Extracción de Entidades
[GoT] Executing Node 4: Validación de Consistencia
[GoT] Executing Node 5: Cálculo de Prioridad
[GoT] Executing Node 6: Determinación de Routing
[GoT] Executing Node 7: Generación

In [75]:
got_execute(TEST_HIDDEN_DATA[:5], "hidden")

Los resultados se guardarán en: results/got_hidden_20251117_181315.csv
Procesando tickets con GoT...

[1/5] Processing ticket...
[GoT] Starting Graph-of-Thoughts execution...
[GoT] Executing Node 1: Análisis de Texto
[GoT] Executing Node 2: Identificación de Intent
[GoT] Executing Node 3: Extracción de Entidades
[GoT] Executing Node 4: Validación de Consistencia
[GoT] Executing Node 5: Cálculo de Prioridad
[GoT] Executing Node 6: Determinación de Routing
[GoT] Executing Node 7: Generación de Action Plan
[GoT] ✓ Ejecución de Grafos completa. Backtracking: No
[1/5] Ticket procesado en 251.96763396263123

[2/5] Processing ticket...
[GoT] Starting Graph-of-Thoughts execution...
[GoT] Executing Node 1: Análisis de Texto
[GoT] Executing Node 2: Identificación de Intent
[GoT] Executing Node 3: Extracción de Entidades
[GoT] Executing Node 4: Validación de Consistencia
[GoT] Executing Node 5: Cálculo de Prioridad
[GoT] Executing Node 6: Determinación de Routing
[GoT] Executing Node 7: Generació